<a href="https://colab.research.google.com/github/rohandekate10/Purdue---Offline-Reinforcement-Learning-Research/blob/main/Solving_Cartpole_Problem_with_d3rlpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Colab and Install d3rlpy

In [ ]:
# Setup Colab

!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# Install d3rlpy
!pip install d3rlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 36.2 MB/s 
     |████████████████████████████████| 58 kB 6.3 MB/s 
     |████████████████████████████████| 125 kB 59.1 MB/s 


In [ ]:
!pip install gym[classic_control]
#suppress pygame's attempt to use a real display device by telling SDL to use a dummy driver
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.3 MB 1.2 MB/s 
pygame 2.1.0 (SDL 2.0.16, Python 3.7.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


<Surface(640x480x32 SW)>

In [ ]:
# Reproducibility
# set random seeds in random module, numpy module and PyTorch module.
import d3rlpy
d3rlpy.seed(100)

# Get Cartpole Dataset

In [ ]:
from d3rlpy.datasets import get_cartpole # CartPole-v0 dataset

In [ ]:
# Create dataset and environment objects
dataset, env = get_cartpole()

/usr/local/lib/python3.7/dist-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: Depre

## Check Dataset

In [ ]:
episode = dataset.episodes[0]

# Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity
episode.observations

array([[ 0.03632258, -0.02921838, -0.03899185, -0.01113326],
       [ 0.03573821, -0.22376007, -0.03921452,  0.26899678],
       [ 0.03126301, -0.02810107, -0.03383458, -0.03579221],
       [ 0.03070099, -0.2227219 , -0.03455043,  0.24602649],
       [ 0.02624655, -0.41733378, -0.0296299 ,  0.5276143 ],
       [ 0.01789988, -0.6120266 , -0.01907761,  0.8108154 ],
       [ 0.00565935, -0.8068821 , -0.0028613 ,  1.0974369 ],
       [-0.0104783 , -1.0019662 ,  0.01908744,  1.3892207 ],
       [-0.03051762, -0.8070872 ,  0.04687185,  1.1025668 ],
       [-0.04665937, -0.6126121 ,  0.06892319,  0.8249499 ],
       [-0.05891161, -0.41849717,  0.08542219,  0.55471617],
       [-0.06728155, -0.22467197,  0.09651651,  0.29012206],
       [-0.07177499, -0.03104928,  0.10231895,  0.02937155],
       [-0.07239597, -0.22747838,  0.10290638,  0.3525035 ],
       [-0.07694554, -0.03395883,  0.10995645,  0.09396093],
       [-0.07762472, -0.23047094,  0.11183567,  0.41921106],
       [-0.08223414, -0.

In [ ]:
episode.observations.shape

(35, 4)

In [ ]:
'''
0: Push cart to the left
1: Push cart to the right
'''

episode.actions

array([0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0], dtype=int32)

In [ ]:
episode.actions.shape

(35,)

In [ ]:
episode.rewards

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.], dtype=float32)

In [ ]:
# Transition
transition = episode.transitions[0]
transition.observation

array([ 0.03632258, -0.02921838, -0.03899185, -0.01113326], dtype=float32)

In [ ]:
transition.action

0

In [ ]:
transition.reward

1.0

In [ ]:
transition.next_observation

array([ 0.03573821, -0.22376007, -0.03921452,  0.26899678], dtype=float32)

# Split Dataset into Training and Test

In [ ]:
# Import library
from sklearn.model_selection import train_test_split
# Split the dataset
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2, random_state=1)

# Setup Discrete Algorithm

## Behavior Cloning algorithm for discrete control

In [ ]:
from gym.spaces import discrete
from d3rlpy.algos import DiscreteBC

discrete_bc = DiscreteBC(use_gpu=True, scaler='standard')

discrete_bc.build_with_dataset(dataset)

discrete_bc

d3rlpy.algos.bc.DiscreteBC(action_scaler=None, batch_size=100, beta=0.5, encoder_factory=d3rlpy.models.encoders.DefaultEncoderFactory(activation='relu', use_batch_norm=False, dropout_rate=None), gamma=1.0, generated_maxlen=100000, impl=<d3rlpy.algos.torch.bc_impl.DiscreteBCImpl object at 0x7fa5a06117d0>, learning_rate=0.001, n_frames=1, n_steps=1, optim_factory=d3rlpy.models.optimizers.AdamFactory(optim_cls='Adam', betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False), real_ratio=1.0, reward_scaler=None, scaler=d3rlpy.preprocessing.scalers.StandardScaler(mean=None, std=None, eps=0.001), use_gpu=d3rlpy.gpu.Device(idx=0))

# Training

## Setup Metrics

In [ ]:
from d3rlpy.metrics.scorer import td_error_scorer
'''Returns average TD error
This metics suggests how Q functions overfit to training sets. 
If the TD error is large, the Q functions are overfitting.'''
#td_error_train = td_error_scorer(discrete_bc, test_episodes) #Error: BC does not support value estimation

from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer 
'''Returns average of discounted sum of advantage.
This metrics suggests how the greedy-policy selects different actions in action-value space. 
If the sum of advantage is small, the policy selects actions with larger estimated action-values.'''
#discounted_sum_advtg_train = discounted_sum_of_advantage_scorer(train_episodes, test_episodes) #Error: 'list' object has no attribute 'n_frames'

from d3rlpy.metrics.scorer import average_value_estimation_scorer
'''Returns average value estimation.
This metrics suggests the scale for estimation of Q functions. 
If average value estimation is too large, the Q functions overestimate action-values, which possibly makes training failed.'''
#avg_val_estimate_train = average_value_estimation_scorer(discrete_bc, test_episodes) #Error: assert self._mean is not None and self._std is not None

from d3rlpy.metrics.scorer import value_estimation_std_scorer
'''Returns standard deviation of value estimation.
This metrics suggests how confident Q functions are for the given episodes. 
This metrics will be more accurate with boostrap enabled and the larger n_critics at algorithm. 
If standard deviation of value estimation is large, the Q functions are overfitting to the training set.'''
#val_estimate_std_train = value_estimation_std_scorer(discrete_bc, train_episodes) #AssertionError:

from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer
'''
Returns mean estimated action-values at the initial states.
This metrics suggests how much return the trained policy would get from the initial states by deploying the policy to the states. 
If the estimated value is large, the trained policy is expected to get higher returns.'''
#init_state_value_estimate_train = initial_state_value_estimation_scorer(discrete_bc, train_episodes) #AssertionError:

from d3rlpy.metrics.scorer import soft_opc_scorer
'''
Returns Soft Off-Policy Classification metrics.
This function returns scorer function, which is suitable to the standard scikit-learn scorer function style. 
The metrics of the scorer funciton is evaluating gaps of action-value estimation between the success episodes and the all episodes. 
If the learned Q-function is optimal, action-values in success episodes are expected to be higher than the others. 
The success episode is defined as an episode with a return above the given threshold.'''
#scorer = soft_opc_scorer(return_threshold=180)

from d3rlpy.metrics.scorer import continuous_action_diff_scorer
'''
Returns squared difference of actions between algorithm and dataset.
This metrics suggests how different the greedy-policy is from the given episodes in continuous action-space. 
If the given episodes are near-optimal, the small action difference would be better.'''

from d3rlpy.metrics.scorer import discrete_action_match_scorer
'''
Returns percentage of identical actions between algorithm and dataset.
This metrics suggests how different the greedy-policy is from the given episodes in discrete action-space. 
If the given episdoes are near-optimal, the large percentage would be better.'''

from d3rlpy.metrics.scorer import evaluate_on_environment
'''Returns scorer function of evaluation on environment.
This function returns scorer function, which is suitable to the standard scikit-learn scorer function style. 
The metrics of the scorer function is ideal metrics to evaluate the resulted policies.'''

from d3rlpy.metrics.comparer import compare_continuous_action_diff
'''Returns scorer function of action difference between algorithms.
This metrics suggests how different the two algorithms are in continuous action-space. 
If the algorithm to compare with is near-optimal, the small action difference would be better.'''

from d3rlpy.metrics.comparer import compare_continuous_action_diff
'''Returns scorer function of action matches between algorithms.
This metrics suggests how different the two algorithms are in discrete action-space. 
If the algorithm to compare with is near-optimal, the small action difference would be better.'''

from d3rlpy.metrics.scorer import dynamics_observation_prediction_error_scorer
'''Returns MSE of observation prediction.
This metrics suggests how dynamics model is generalized to test sets. 
If the MSE is large, the dynamics model are overfitting.'''

from d3rlpy.metrics.scorer import dynamics_reward_prediction_error_scorer
'''Returns MSE of reward prediction.
This metrics suggests how dynamics model is generalized to test sets. 
If the MSE is large, the dynamics model are overfitting.'''

from d3rlpy.metrics.scorer import dynamics_prediction_variance_scorer
'''Returns prediction variance of ensemble dynamics.
This metrics suggests how dynamics model is confident of test sets. 
If the variance is large, the dynamics model has large uncertainty.'''

'Returns prediction variance of ensemble dynamics.\nThis metrics suggests how dynamics model is confident of test sets. \nIf the variance is large, the dynamics model has large uncertainty.'

In [ ]:
all_scorers={'td_error' : td_error_scorer,
             'discounted_sum_of_advantage': discounted_sum_of_advantage_scorer,
             'average_value_estimation' : average_value_estimation_scorer,
             'value_estimation_std':value_estimation_std_scorer,
             'initial_state_value_estimation':initial_state_value_estimation_scorer,
             'soft_opc' :soft_opc_scorer(return_threshold=180),
             'continuous_action_diff':continuous_action_diff_scorer,
             'discrete_action_match':discrete_action_match_scorer,
             'evaluate_on_environment': evaluate_on_environment(env, render = True),
             'compare_continuous_action_diff': compare_continuous_action_diff(discrete_bc),
             #'compare_continuous_action_diff': compare_continuous_action_diff(discrete_bc),
             'dynamics_observation_prediction_error':dynamics_observation_prediction_error_scorer,
             'dynamics_reward_prediction_error':dynamics_reward_prediction_error_scorer}
all_scorers

{'td_error': <function d3rlpy.metrics.scorer.td_error_scorer(algo: d3rlpy.metrics.scorer.AlgoProtocol, episodes: List[d3rlpy.dataset.Episode]) -> float>,
 'discounted_sum_of_advantage': <function d3rlpy.metrics.scorer.discounted_sum_of_advantage_scorer(algo: d3rlpy.metrics.scorer.AlgoProtocol, episodes: List[d3rlpy.dataset.Episode]) -> float>,
 'average_value_estimation': <function d3rlpy.metrics.scorer.average_value_estimation_scorer(algo: d3rlpy.metrics.scorer.AlgoProtocol, episodes: List[d3rlpy.dataset.Episode]) -> float>,
 'value_estimation_std': <function d3rlpy.metrics.scorer.value_estimation_std_scorer(algo: d3rlpy.metrics.scorer.AlgoProtocol, episodes: List[d3rlpy.dataset.Episode]) -> float>,
 'initial_state_value_estimation': <function d3rlpy.metrics.scorer.initial_state_value_estimation_scorer(algo: d3rlpy.metrics.scorer.AlgoProtocol, episodes: List[d3rlpy.dataset.Episode]) -> float>,
 'soft_opc': <function d3rlpy.metrics.scorer.soft_opc_scorer.<locals>.scorer(algo: d3rlpy.me

## Tensorboard Setup

In [ ]:
# Setup TensorBoard
%load_ext tensorboard
import tensorflow as tf
import datetime

## Offline Training

### Unsupported Metrics by Discrete BC Algorithm
1. td_error_scorer
2. discounted_sum_of_advantage_scorer
3. average_value_estimation_scorer
4. value_estimation_std_scorer
5. initial_state_value_estimation_scorer
6. soft_opc_scorer
7. continuous_action_diff_scorer
8. compare_continuous_action_diff

In [ ]:
history = discrete_bc.fit(train_episodes,
                          n_epochs=100,
                          verbose=True,
                          eval_episodes=test_episodes,
                          scorers={
                              'evaluate_on_environment': evaluate_on_environment(env, render = True),
                              'discrete_action_match':discrete_action_match_scorer,
                          },
                          tensorboard_dir='discrete_bc_run'
                          )

2022-11-01 19:35.09 [debug    ] RoundIterator is selected.
2022-11-01 19:35.09 [info     ] Directory is created at d3rlpy_logs/DiscreteBC_20221101193509
2022-11-01 19:35.09 [debug    ] Fitting scaler...              scaler=standard
2022-11-01 19:35.09 [warning  ] Skip building models since they're already built.
2022-11-01 19:35.09 [info     ] Parameters are saved to d3rlpy_logs/DiscreteBC_20221101193509/params.json params={'action_scaler': None, 'batch_size': 100, 'beta': 0.5, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.001, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': {'type': 'standard', 'params': {'mean': array([[-0.0264398 , -0.05434881, -0.00895111, -0.01157843]]), 'std': array([[0.68274691, 0.881

Epoch 1/100:   0%|          | 0/802 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/gym/core.py:50: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  "You are calling render method, "


2022-11-01 19:35.56 [info     ] DiscreteBC_20221101193509: epoch=1 step=802 epoch=1 metrics={'time_sample_batch': 0.00023473141496615517, 'time_algorithm_update': 0.00794824757183579, 'loss': 0.6691344367001122, 'time_step': 0.008240814518155599, 'evaluate_on_environment': 198.8, 'discrete_action_match': 0.6302339901477833} step=802
2022-11-01 19:35.56 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteBC_20221101193509/model_802.pt


Epoch 2/100:   0%|          | 0/802 [00:00<?, ?it/s]

...

2022-11-01 20:38.18 [info     ] DiscreteBC_20221101193509: epoch=98 step=78596 epoch=98 metrics={'time_sample_batch': 0.00025922728892870974, 'time_algorithm_update': 0.0025339640881355267, 'loss': 0.6558461229402823, 'time_step': 0.002867500086377684, 'evaluate_on_environment': 197.3, 'discrete_action_match': 0.6439347290640394} step=78596
2022-11-01 20:38.18 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteBC_20221101193509/model_78596.pt


Epoch 99/100:   0%|          | 0/802 [00:00<?, ?it/s]

2022-11-01 20:39.01 [info     ] DiscreteBC_20221101193509: epoch=99 step=79398 epoch=99 metrics={'time_sample_batch': 0.0002442842112515038, 'time_algorithm_update': 0.0024947912021170827, 'loss': 0.6557910617004309, 'time_step': 0.002806057656495054, 'evaluate_on_environment': 200.0, 'discrete_action_match': 0.6475266830870279} step=79398
2022-11-01 20:39.01 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteBC_20221101193509/model_79398.pt


Epoch 100/100:   0%|          | 0/802 [00:00<?, ?it/s]

2022-11-01 20:39.44 [info     ] DiscreteBC_20221101193509: epoch=100 step=80200 epoch=100 metrics={'time_sample_batch': 0.00025774861809024194, 'time_algorithm_update': 0.0025685217017842052, 'loss': 0.6556123622932339, 'time_step': 0.0029035693689474736, 'evaluate_on_environment': 200.0, 'discrete_action_match': 0.6486042692939245} step=80200
2022-11-01 20:39.44 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteBC_20221101193509/model_80200.pt


In [ ]:
%tensorboard --logdir discrete_bc_run

<IPython.core.display.Javascript object>